In [ ]:
!pip install yfinance
!pip install -q --upgrade ipython
!pip install -q --upgrade ipykernel
!pip install ta
!pip install backtrader
!pip install datetime

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'
from datetime import datetime
import io as io
import random as random
import hyperopt as hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope
from functools import partial
import pandas_datareader as pandas_datareader
#to visualize the results
import seaborn as sns
from pandas_datareader import data as pdr
# add technical analysis features
from ta.trend import macd
from ta.trend import macd_signal
from ta.trend import macd_diff
from ta.volatility import bollinger_wband
from ta.trend import sma_indicator
from ta.momentum import rsi
from ta.momentum import stochrsi
from ta.volatility import bollinger_pband
from ta.volume import volume_weighted_average_price
from ta.volatility import average_true_range

import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)
%load_ext google.colab.data_table

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
colnames=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'] 
main_df = pd.read_csv("/content/drive/MyDrive/Trading/ETF Data Sets/QQQ.txt", names=colnames, header=None)
main_df['Date'] = tqdm(pd.to_datetime(main_df['Date']))

100%|██████████| 3354500/3354500 [00:07<00:00, 476776.36it/s]


In [ ]:
# Time Slicer

# First 2 minutes of the morning
subsetDataFrame = main_df[main_df['Time'] == '09:31']

# # 5 minute slice
# main_df2 = main_df
# _5min_df = main_df[::5]

In [ ]:
filtered_Close = main_df2.loc[(main_df2['Date/Time'] >= '2020-10-29')
                     & (main_df2['Date/Time'] < '2021-11-09')]

In [ ]:
filtered_Close = main_df.loc[(main_df['Date'] >= '2011-11-09')
                     & (main_df['Date'] < '2021-11-09')]

In [ ]:
 #list1 = set(main_df2['Time'])
 for v in tqdm(range(0,len(list1))): 
  filtered_Close = filtered_Close.loc[~((filtered_Close['Time'] == list1[v]))]

100%|██████████| 596/596 [00:01<00:00, 333.04it/s]


In [ ]:
MA_Length = 30 # 79
BBW_Threshold = 15 # 69.82
# BBp_Threshold = 7
# MA_Length = params['MA_Length']
# BBW_Threshold = params['BBW_Threshold']
# Signal_Threshold = params['Signal_Threshold']
MACD_Length = 20
RSI_Threshold_BUY = 60
RSI_Threshold_SELL = 8
Inital_Capital = 10000

# Initializes data to the Close dataframe
Close = _5min_df[['Date','Time','Open', 'Close', 'Volume', 'High', 'Low']]

# Calculates the Moving Average
Close["SMA"] = sma_indicator(Close.Close,  window=MA_Length)
Close['MACD'] = macd(Close.Close)
Close['BBW'] = 10*bollinger_wband(Close.Close, 20, 2)
Close['SIGNAL'] = macd_signal(Close.Close)
Close['RSI'] = 100*stochrsi(Close.Close)
Close['BB%'] = 100*bollinger_pband(Close.Close)
Close['VWAP'] = volume_weighted_average_price(Close.High, Close.Low, Close.Close, Close.Volume)
Close['ATR'] = 3*average_true_range(Close.High,Close.Low, Close.Close)
Close['High_ATR_Price'] = Close.Close + Close.ATR
Close['Low_ATR_Price'] = Close.Close - Close.ATR
Close['Buy'] = np.where(
                      # (Close['SMA'] < Close['Close']) &  
                      # (Close['SMA'] < Close['VWAP'])
                      (Close.Close > Close.High_ATR_Price.shift(1))
                      # (Close['MACD'] < 0) &
                      # (Close['SIGNAL'] > Signal_Threshold) &
                      # (Close['BBW'] > BBW_Threshold) &
                      # (Close['BB%'] < BBp_Threshold)
                      # (Close['RSI'] < RSI_Threshold_BUY)
                      , 1, 0)

Close['Sell'] = np.where(
                        # (Close['SMA'] > Close['Close']) & 
                        # (Close['SMA'] > Close['VWAP'])
                        (Close.Close < Close.Low_ATR_Price.shift(1))
                        # (Close['MACD'] > 0) &
                        # (Close['SIGNAL'] > Signal_Threshold) &
                        # (Close['BBW'] > BBW_Threshold) &
                        # (Close['BB%'] > BBp_Threshold)
                        # (Close['RSI'] > RSI_Threshold_SELL)
                        , 1, 0)

Close['Buyprice'] = Close.Close
Close['Sellprice'] = Close.Close

x = Close[(Close.Buy == 1) | (Close.Sell == 1)]
y = x[(x.Buy.diff() == 1) | x.Sell.diff() == 1]
profits = ((y.Sellprice.shift(-1) - y.Buyprice)*100)
profits = profits[::2]
cumprofits = np.cumsum(profits) + Inital_Capital

Total_Profits = round(profits.sum(),0)
# Hold_Period_Profits = round((Close.iloc[0,2] - Close.iloc[-2,3])*-100,0)

def Hold_Profits(startprice, endprice):
  Hold_Period_Profits = round((startprice - endprice)*-100,0)

  if Hold_Period_Profits < 7000:
    Hold_Period_Profits = 7000
  else:
    Hold_Period_Profits
  return Hold_Period_Profits

Hold_Period_Profits = 7000
Hold_Period_Profits = Hold_Profits(Close.iloc[0,2],Close.iloc[-2,3])

Profit_Diff = round(Total_Profits-Hold_Period_Profits,0)
Benefit_Ratio = round(Total_Profits/Hold_Period_Profits,4)
Ave_Profits = round(profits.mean(),4)

num_years = 23
trading_days_in_a_year = 252
total_trading_days = num_years*trading_days_in_a_year

print('Total Profits ' + str(Total_Profits))
print('Hold Period Profits ' + str(Hold_Period_Profits))
print('Hold/Total Diff ' + str(Total_Profits-Hold_Period_Profits))
print('Average Profits ' + str(Ave_Profits))
# print('Hold Period Return ' + str(Hold_Period_Return))
print('Benefit Ratio ' + str(float(Benefit_Ratio)))
print('# of Trades ' + str(len(profits)))
print('# of Trades/Year ' + str(len(profits)/num_years))
# Close.to_csv("Close.csv")

Total Profits 21996.0
Hold Period Profits 31547.0
Hold/Total Diff -9551.0
Average Profits 14.625
Benefit Ratio 0.6972
# of Trades 1505
# of Trades/Year 65.43478260869566


In [ ]:
print('BBW min: ' + str(Close['BBW'].min()))
print('BBW max: ' + str(Close['BBW'].max()))
print('BBW mean: ' + str(Close['BBW'].mean()))
print('BBW median: ' + str(Close['BBW'].median()))

In [ ]:
HyperOpt_space = {
            # 'BBW_Threshold': scope.int(hp.quniform('BBW_Threshold',5,150,1)),
            'MA_Length': scope.int((hp.randint('MA_Length',300))),
            # 'VWAP_Window': scope.int((hp.quniform('VWAP_Window',5,250,1))),
            # 'BBW_MA_Length': scope.int(hp.quniform('BBW_MA_Length',5,250,1)),
            # 'Slow_MA': scope.int(hp.quniform('Slow_MA',5,250,1)),
            # 'Fast_MA': scope.int(hp.quniform('Fast_MA',5,250,1)),
            # 'Signal_MA': scope.int(hp.quniform('Signal_MA',5,250,1)),
            # 'Signal_Threshold1': scope.int(hp.quniform('Signal_Threshold1',5,50,1)),
            # 'Signal_Threshold2': scope.int(hp.quniform('Signal_Threshold2',5,50,1)),
            # 'MACD_Length': scope.int(hp.randint('MACD_Length',1000)),
            # 'BBp_Threshold': scope.int(hp.randint('BBp_Threshold',1000)),
            'RSI_Threshold_BUY': scope.int(hp.randint('RSI_Threshold_BUY',100)),
            'RSI_Threshold_SELL': scope.int(hp.randint('RSI_Threshold_SELL',100))
            # 'TimeFrame': scope.int(hp.quniform('TimeFrame',1,60,1))
            # 'ATR_Muliple': scope.int(hp.randint('ATR_Muliple',15))
        }

In [ ]:
def backtest3(params):


  #--DECLARED VARIABLES--#
  #|||||||||||||||||||||||#
  MA_Length = params['MA_Length']
  BBW_Threshold = 146 #params['BBW_Threshold']
  # Signal_Threshold1 = params['Signal_Threshold1']
  # Signal_Threshold2 = params['Signal_Threshold2']
  # MACD_Length = params['MACD_Length']
  RSI_Threshold_BUY = params['RSI_Threshold_BUY']
  RSI_Threshold_SELL = params['RSI_Threshold_SELL']
  # BBp_Threshold = params['BBp_Threshold']
  VWAP_Window = 20 #params['VWAP_Window']
  # ATR_Muliple = params['ATR_Muliple']
  Inital_Capital = 10000
  # _5min_df = main_df[::params['TimeFrame']]
  

  #--DATA SOURCE--#
  # Close = _5min_df[['Date','Time','Open', 'Close', 'Volume', 'High', 'Low']]
  Close = df


  #--CALCULATES THE TECHINCAL INDICATORS--#
  Close["SMA"] = sma_indicator(Close.Close,  window=MA_Length)
  Close['MACD'] = macd(Close.Close)
  Close['BBW'] = 100*bollinger_wband(Close.Close, 20, 2)
  Close['SIGNAL'] = 1000*macd_signal(Close.Close)
  Close['RSI'] = 100*stochrsi(Close.Close)
  Close['BB%'] = 100*bollinger_pband(Close.Close)
  Close['VWAP'] = volume_weighted_average_price(Close.High, Close.Low, Close.Close, Close.Volume,window=VWAP_Window)
  # Close['ATR'] = ATR_Muliple*average_true_range(Close.High,Close.Low, Close.Close)
  Close['High_ATR_Price'] = Close.Close + Close.ATR
  Close['Low_ATR_Price'] = Close.Close - Close.ATR

  #--Defines the Buy and Sell Flags--#
  Close['Buy'] = np.where(
                        # (Close['SMA'] > Close['Close']) &
                        (Close['SMA'].diff() > Close['SMA']) &
                        # (Close['SMA'] < Close['VWAP'])
                        # (Close.Close > Close.High_ATR_Price.shift(1))
                        # (Close['VWAP'] > Close['Close'])
                        # (Close['MACD'] < 0) &
                        # (Close['SIGNAL'] > Signal_Threshold1) &
                        # (Close['BBW'] > BBW_Threshold) &
                        # (Close['BB%'] < BBp_Threshold)
                        (Close['RSI'] < RSI_Threshold_BUY)
                        , 1, 0)

  Close['Sell'] = np.where(
                          # (Close['SMA'] < Close['Close']) &
                          (Close['SMA'].diff() < Close['SMA']) &
                          #  (Close['SMA'] > Close['VWAP'])
                          # (Close.Close < Close.Low_ATR_Price.shift(1))
                          #  (Close['VWAP'] < Close['Close'])
                          # (Close['MACD'] > 0) &
                          # (Close['SIGNAL'] > Signal_Threshold2) &
                          # (Close['BBW'] > BBW_Threshold) &
                          # (Close['BB%'] > BBp_Threshold)
                          (Close['RSI'] > RSI_Threshold_SELL)
                          , 1, 0)
  

  #--Grabs the buy/sell price--#
  Close['Buyprice'] = Close.Close
  Close['Sellprice'] = Close.Close


  #--Calculates the Profits--#
  x = Close[(Close.Buy == 1) | (Close.Sell == 1)]
  y = x[(x.Buy.diff() == 1) | x.Sell.diff() == 1]
  profits = ((y.Sellprice.shift(-1) - y.Buyprice)*(100))
  cumprofits = profits + Inital_Capital
  profits = profits[::2]


  #--Metrics--#
  Total_Profits = profits.sum()
  Hold_Period_Profits = (Close.iloc[0,2] - Close.iloc[-2,3])*-100
  # Hold_Period_Profits = 7000
  Profit_Diff = Total_Profits-Hold_Period_Profits
  Benefit_Ratio = round(Total_Profits/Hold_Period_Profits,4)
  Ave_Profits = round(profits.mean(),2)
  ROI = round(Total_Profits/Inital_Capital,2)*100
  
  #--Objective--#
  Loss_cat = -Benefit_Ratio

  
  #--Returns What is Needed the Run the HyperOpt--#
  return {'loss': Loss_cat, 'status': STATUS_OK,'Trained_Model': Close}

best_models = [[]for i in range(20)]

#--Runs the HyperOpt--#
for tests in range(0,20):
  trials = Trials()
  best_model = fmin(partial(backtest3),HyperOpt_space,algo=tpe.suggest,max_evals=50,trials=trials)
  print(best_model)
  best_score = trials.best_trial['result']['loss']
  best_models[0].append(best_model)
  best_models[1].append(best_score)





100%|██████████| 50/50 [00:01<00:00, 30.23it/s, best loss: -0.0]
{'MA_Length': 6, 'RSI_Threshold_BUY': 83, 'RSI_Threshold_SELL': 83}
100%|██████████| 50/50 [00:01<00:00, 34.93it/s, best loss: -0.0]
{'MA_Length': 100, 'RSI_Threshold_BUY': 76, 'RSI_Threshold_SELL': 35}
  8%|▊         | 4/50 [00:00<00:01, 36.35it/s, best loss: -0.0]

In [ ]:
best_models = pd.DataFrame(best_models)

In [ ]:
# SPY

# 5 min, filtered

#Best stochrsi
#100%|██████████| 100/100 [00:04<00:00, 23.44it/s, best loss: -1.0098]
#{'RSI_Threshold_BUY': 30, 'RSI_Threshold_SELL': 17}

#Best MA + RSI
#100%|██████████| 1000/1000 [00:54<00:00, 18.31it/s, best loss: -1.2769]
#{'MA_Length': 3, 'RSI_Threshold_BUY': 28, 'RSI_Threshold_SELL': 30}

#Best MA + RSI + BBW
#100%|██████████| 200/200 [00:10<00:00, 19.14it/s, best loss: -1.3011]
#{'BBW_Threshold': 44, 'MA_Length': 3, 'RSI_Threshold_BUY': 29, 'RSI_Threshold_SELL': 48}


# 5 min, unfiltered

# 100%|██████████| 20/20 [00:00<00:00, 22.27it/s, best loss: -1.4017]
# {'BBW_Threshold': 54, 'MA_Length': 3}


#5 min, kibot data, full 

# 100%|██████████| 105/105 [00:40<00:00,  2.61it/s, best loss: -1.472]
# {'BBW_Threshold': 3, 'MA_Length': 6, 'RSI_Threshold_BUY': 91, 'RSI_Threshold_SELL': 15}

# /\
# ||
# Appears to work very well

# 100%|██████████| 105/105 [00:40<00:00,  2.57it/s, best loss: -1.3261]
# {'BBW_Threshold': 6, 'MA_Length': 25, 'RSI_Threshold_BUY': 61, 'RSI_Threshold_SELL': 97}

# 100%|██████████| 20/20 [00:00<00:00, 22.27it/s, best loss: -1.7418]
# {'BBW_Threshold': 3, 'MA_Length': 10}

In [ ]:
# XLE

# 5 min, kibot data, full

# 100%|██████████| 150/150 [00:56<00:00,  2.65it/s, best loss: -2.0748]
# {'BBW_Threshold': 11, 'MA_Length': 85, 'RSI_Threshold_BUY': 32, 'RSI_Threshold_SELL': 2}
# 100%|██████████| 150/150 [00:56<00:00,  2.64it/s, best loss: -2.7666]
# {'BBW_Threshold': 7, 'MA_Length': 3, 'RSI_Threshold_BUY': 49, 'RSI_Threshold_SELL': 27}
# 100%|██████████| 150/150 [00:57<00:00,  2.63it/s, best loss: -1.8051]
# {'BBW_Threshold': 12, 'MA_Length': 96, 'RSI_Threshold_BUY': 59, 'RSI_Threshold_SELL': 6}
# 100%|██████████| 150/150 [00:56<00:00,  2.64it/s, best loss: -2.337]
# {'BBW_Threshold': 4, 'MA_Length': 3, 'RSI_Threshold_BUY': 23, 'RSI_Threshold_SELL': 69}
# 100%|██████████| 150/150 [00:56<00:00,  2.65it/s, best loss: -1.6251]
# {'BBW_Threshold': 11, 'MA_Length': 94, 'RSI_Threshold_BUY': 63, 'RSI_Threshold_SELL': 44}
# 100%|██████████| 50/50 [00:19<00:00,  2.60it/s, best loss: -1.76]
# {'BBW_Threshold': 6, 'MA_Length': 153.0, 'RSI_Threshold_BUY': 89, 'RSI_Threshold_SELL': 70}
# 100%|██████████| 50/50 [00:19<00:00,  2.62it/s, best loss: -2.0916]
# {'BBW_Threshold': 1, 'MA_Length': 6.0, 'RSI_Threshold_BUY': 94, 'RSI_Threshold_SELL': 46}
# 100%|██████████| 50/50 [00:19<00:00,  2.62it/s, best loss: -1.9234]
# {'BBW_Threshold': 4, 'MA_Length': 44.0, 'RSI_Threshold_BUY': 16, 'RSI_Threshold_SELL': 87}
# 100%|██████████| 50/50 [00:19<00:00,  2.57it/s, best loss: -1.8182]
# {'BBW_Threshold': 8, 'MA_Length': 116.0, 'RSI_Threshold_BUY': 76, 'RSI_Threshold_SELL': 15}

In [ ]:
# EWZ

# 5 min, kibot, full data

# 100%|██████████| 50/50 [00:18<00:00,  2.76it/s, best loss: -35.9733]
# {'BBW_Threshold': 19, 'MA_Length': 50.0, 'RSI_Threshold_BUY': 43, 'RSI_Threshold_SELL': 13}

# 100%|██████████| 50/50 [00:18<00:00,  2.72it/s, best loss: -1.1627]
# {'BBW_Threshold': 19, 'MA_Length': 46.0, 'RSI_Threshold_BUY': 75, 'RSI_Threshold_SELL': 7}

In [ ]:
# GLD

# 5 min, kibot, full range

# 100%|██████████| 50/50 [00:15<00:00,  3.18it/s, best loss: -2.0686]
# {'BBW_Threshold': 21, 'MA_Length': 118.0, 'RSI_Threshold_BUY': 55, 'RSI_Threshold_SELL': 76}
# 100%|██████████| 50/50 [00:15<00:00,  3.17it/s, best loss: -1.9084]
# {'BBW_Threshold': 17, 'MA_Length': 71.0, 'RSI_Threshold_BUY': 65, 'RSI_Threshold_SELL': 93}
# 100%|██████████| 50/50 [00:15<00:00,  3.19it/s, best loss: -2.0088]
# {'BBW_Threshold': 20, 'MA_Length': 246.0, 'RSI_Threshold_BUY': 52, 'RSI_Threshold_SELL': 2}
# 100%|██████████| 50/50 [00:15<00:00,  3.16it/s, best loss: -1.8881]
# {'BBW_Threshold': 19, 'MA_Length': 247.0, 'RSI_Threshold_BUY': 75, 'RSI_Threshold_SELL': 22}
# 100%|██████████| 50/50 [00:15<00:00,  3.18it/s, best loss: -1.9204]
# {'BBW_Threshold': 19, 'MA_Length': 9.0, 'RSI_Threshold_BUY': 87, 'RSI_Threshold_SELL': 12}
# 100%|██████████| 250/250 [01:58<00:00,  2.12it/s, best loss: -3.5393]
# {'BBW_Threshold': 10, 'MA_Length': 34.0, 'RSI_Threshold_BUY': 78, 'RSI_Threshold_SELL': 44}

In [ ]:
# PFE

# 5 min, kibot, full range

# 100%|██████████| 250/250 [02:35<00:00,  1.61it/s, best loss: -25.8984]
# {'BBW_Threshold': 5.0, 'MA_Length': 5.0, 'RSI_Threshold_BUY': 93, 'RSI_Threshold_SELL': 6}